## データローダーの実装

In [ ]:
import zipfile
from logging import Logger, getLogger
from os import mkdir
from typing import Callable
from urllib.parse import urlparse
from zipfile import BadZipFile

import numpy as np
import requests
from jupyterthemes.stylefx import check_directories
from requests import Response
from torch import no_grad
from torch.autograd import Variable
from torchvision.transforms.v2 import ToTensor, RandomHorizontalFlip
from tqdm import tqdm

KIB = 2 ** 10


class Downloader:
    """
    指定されたURLからデータセットをダウンロードし、ZIPファイルを解凍するクラス。

    :param root: ダウンロード先のルートディレクトリ。
    :param url: ダウンロードするURLまたはURLを返す関数。
    :param overwrite: 既存のファイルを上書きするかどうか（デフォルトはFalse）。
    :param zip_filename: ZIPファイルの名前（指定がない場合はURLから取得）。
    :param logger: ロギング用のLoggerオブジェクト（指定がない場合はデフォルトのLoggerを使用）。
    """

    def __init__(self, root: str, url: str | Callable[[], str] = None, overwrite: bool = False,
                 zip_filename: str = None,
                 logger: Logger = None):
        """
        Downloaderのコンストラクタ。

        :param root: ダウンロード先のルートディレクトリ。
        :param url: ダウンロードするURLまたはURLを返す関数。
        :param overwrite: 既存のファイルを上書きするかどうか（デフォルトはFalse）。
        :param zip_filename: ZIPファイルの名前（指定がない場合はURLから取得）。
        :param logger: ロギング用のLoggerオブジェクト（指定がない場合はデフォルトのLoggerを使用）。
        """
        self._root = os.path.normpath(root)

        if isinstance(url, Callable):
            self.url = url()
        else:
            self.url = url

        self.zip_filename = zip_filename or os.path.basename(urlparse(self.url).path)  # URLからファイル名を取得
        self.zip_path = os.path.join(self._root, self.zip_filename)
        self.extract_path = os.path.splitext(self.zip_path)[0]
        self.overwrite = overwrite
        self.logger = logger or getLogger(__name__)

    def download(self):
        """データセットをダウンロードする。

        既にデータセットがダウンロードされている場合、overwriteがFalseの場合はダウンロードをスキップします。
        """
        if self.is_downloaded() and not self.overwrite:
            print(f"Dataset already exists at {self.zip_path}, skipping download.")
            return

        print(f"Downloading dataset from {self.url}...")

        try:
            response = self.request(self.url)
            self.save_response_content(response, self.zip_path)

            print("\nDownload completed.")

        except requests.exceptions.RequestException as e:
            print(f"Error during download: {e}")

        except zipfile.BadZipFile:
            print("Error: Bad zip file.")

    def request(self, url):
        """指定されたURLにGETリクエストを送り、レスポンスを返す。

        :param url: リクエストするURL。
        :return: リクエストの結果得られたレスポンスオブジェクト。
        """
        response = requests.get(url, stream=True)
        response.raise_for_status()  # HTTPエラーを確認
        return response

    def save_response_content(self, response: Response, destination, chunk_size: int = 100 * KIB):
        """レスポンスのコンテンツを指定されたファイルに保存する。

        :param response: 保存するためのレスポンスオブジェクト。
        :param destination: 保存先ファイルのパス。
        :param chunk_size: 保存時のチャンクサイズ（デフォルトは100KiB）。
        """
        if not exists(self.root):
            mkdir(self.root)

        with open(destination, "wb") as f:
            for chunk in tqdm(response.iter_content(chunk_size=chunk_size)):
                if chunk:
                    f.write(chunk)

    def extract(self):
        """ZIPファイルを解凍し、重複したルートフォルダがある場合はまとめる。

        解凍先のディレクトリが既に存在する場合、その内容は保持されます。
        """
        print(f"Unzipping {self.zip_path}...")
        try:
            with zipfile.ZipFile(self.zip_path, 'r') as zip_ref:
                # ZIPファイルのトップレベルのフォルダを確認
                top_level_dirs = {os.path.normpath(x).split(os.sep)[0] for x in zip_ref.namelist()}

                if len(top_level_dirs) == 1:
                    # トップレベルに1つのディレクトリだけある場合
                    top_level_dir = next(iter(top_level_dirs))
                    self.extract_path = os.path.join(self._root, top_level_dir)
                    print(f"Extracting into {self.extract_path}...")

                total_files = len(zip_ref.namelist())
                with tqdm(total=total_files, unit='file') as progress:
                    for file in zip_ref.namelist():
                        destination = self._root if len(top_level_dirs) == 1 else self.extract_path
                        zip_ref.extract(file, destination)
                        progress.update(1)
        except BadZipFile:
            print("Error: Bad zip file, extraction failed.")

        print(f"Extracted to {self._root}.")

    @property
    def root(self):
        """ダウンロード先のルートディレクトリを取得する。"""
        return self._root

    def is_downloaded(self):
        """ZIPファイルがダウンロードされているかどうかを確認する。

        :return: ZIPファイルが存在する場合はTrue、それ以外はFalse。
        """
        return os.path.exists(self.zip_path)

    def is_extracted(self):
        """データセットが解凍されているかどうかを確認する。

        :return: 解凍先が存在する場合はTrue、それ以外はFalse。
        """
        return os.path.exists(self.extract_path)

    def __call__(self, on_complete: Callable = None):
        """ダウンロードおよび解凍を実行する。

        :param on_complete: 処理完了後に呼び出す関数（オプション）。
        """
        if self.overwrite or not self.is_downloaded():
            self.download()
        else:
            print("Dataset exists and 'overwrite' is False. No download.")

        if self.overwrite or not self.is_extracted():
            self.extract()
        else:
            print("Dataset exists and 'overwrite' is False. No extract.")

        if on_complete is not None:
            on_complete()

In [ ]:
import os
from typing import Optional

from torch.utils.data import Dataset, DataLoader, ConcatDataset

from datasets.downloader import Downloader


class Cat2000(Dataset):
    def __init__(self, categories: Optional[list[str]] = None,
                 image_transform: Optional = None,
                 map_transform: Optional = None,
                 downloader: Optional[Downloader] = None,
                 ):
        if categories is None:
            categories = ["*"]
        self.categories = categories
        self.image_transform = image_transform
        self.map_transform = map_transform
        self.downloader = downloader or Downloader("./data", "http://saliency.mit.edu/trainSet.zip")
        self.dataset_path = os.path.join(self.downloader.root, "trainSet", "Stimuli")

        # 画像とマップのペアを取得
        self.image_map_pair_cache = []
        self.downloader(on_complete=self.cache_image_map_paths)

    def cache_image_map_paths(self):
        self.image_map_pair_cache = []

        # categoriesにワイルドカードが含まれている場合、全カテゴリディレクトリを展開
        if "*" in self.categories:
            expanded_categories = [d for d in glob.glob(os.path.join(self.dataset_path, "*")) if os.path.isdir(d)]
        else:
            expanded_categories = [os.path.join(self.dataset_path, category) for category in self.categories]

        # 展開したカテゴリディレクトリごとに処理を行う
        for category_path in expanded_categories:
            # 画像ファイルのパスを取得
            image_paths = glob.glob(os.path.join(category_path, "*.jpg"))

            for image_path in image_paths:
                base_name = os.path.basename(image_path)
                map_name = base_name.replace(".jpg", "_SaliencyMap.jpg")
                map_path = os.path.join(category_path, "Output", map_name)

                if os.path.exists(map_path):
                    self.image_map_pair_cache.append((image_path, map_path))
                else:
                    print(f"Warning: No corresponding map found for {map_path}")

    def __len__(self):
        return len(self.image_map_pair_cache)

    def __getitem__(self, idx: int):
        image_path, map_path = self.image_map_pair_cache[idx]
        image = Image.open(image_path).convert("RGB")
        map_image = Image.open(map_path).convert("RGB")

        if self.image_transform is not None:
            image = self.image_transform(image)

            if self.map_transform is not None:
                map_image = self.map_transform(map_image)
            else:
                map_image = self.image_transform(map_image)

        return image, map_image

    def __str__(self):
        return "\n".join(
            f"image: {Image.open(pair[0]).size}, map: {Image.open(pair[1]).size}" for pair in self.image_map_pair_cache)


In [ ]:
import glob
from os import path
from typing import Optional, Callable

from PIL import Image
from torch.utils.data import Dataset

from datasets.downloader import Downloader


class SALICONDataset(Dataset):

    def __init__(self,
                 val_mode: bool = False,
                 image_transform: Optional[Callable] = None,
                 map_transform: Optional[Callable] = None,
                 images_downloader: Optional[Downloader] = None,
                 map_downloader: Optional[Downloader] = None,
                 ):

        self.categories = "val" if val_mode else "train"

        self.image_transform = image_transform
        self.map_transform = map_transform

        self.images_downloader = images_downloader or Downloader("./data/salicon", "", zip_filename="images.zip",
                                                                 overwrite=False)
        self.maps_downloader = map_downloader or Downloader("./data/salicon", "", zip_filename="maps.zip",
                                                            overwrite=False)

        self.images_downloader()
        self.maps_downloader()

        # 画像とマップのペアを取得
        self.image_map_pair_cache = []
        self.cache_image_map_paths()

    def cache_image_map_paths(self):
        for category in self.categories:
            images_dir = self.images_downloader.extract_path
            maps_dir = self.maps_downloader.extract_path

            images_path_list = sorted(glob.glob(path.join(images_dir, category, "*.jpg")))
            maps_path_list = sorted(glob.glob(path.join(maps_dir, category, "*.png")))

            # ペアリング
            for img_path, map_path in zip(images_path_list, maps_path_list):
                if path.basename(img_path) == path.basename(map_path).replace(".png", ".jpg"):
                    self.image_map_pair_cache.append((img_path, map_path))

    def __len__(self):
        return len(self.image_map_pair_cache)

    def __getitem__(self, index: int):
        image_path, map_path = self.image_map_pair_cache[index]
        image = Image.open(image_path).convert("RGB")
        map_image = Image.open(map_path).convert("RGB")

        if self.image_transform is not None:
            image = self.image_transform(image)

            if self.map_transform is not None:
                map_image = self.map_transform(map_image)
            else:
                map_image = self.image_transform(map_image)

        return image, map_image

    def __str__(self):
        return "\n".join(
            f"image: {Image.open(pair[0]).size}, map: {Image.open(pair[1]).size}" for pair in self.image_map_pair_cache)


## モデルの実装

In [1]:
from torch.nn import Module
from torchvision.models import vgg16


class DecoderBlock(Module):
    def __init__(self, in_channels=512, out_channels=512):
        super().__init__()
        self.module = Sequential(
            Conv2d(in_channels, out_channels, 3, padding=1),
            LeakyReLU()
        )


class Generator(Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.encoder1 = vgg16(pretrained=pretrained).features[:17]
        self.encoder_last = vgg16(pretrained=pretrained).features[17:-1]
        self.decoder = Sequential(
            DecoderBlock(512, 512),
            DecoderBlock(512, 512),
            DecoderBlock(512, 512),
            Upsample(scale_factor=2),

            DecoderBlock(512, 512),
            DecoderBlock(512, 512),
            DecoderBlock(512, 512),

            Upsample(scale_factor=2),

            DecoderBlock(512, 256),
            DecoderBlock(256, 256),
            DecoderBlock(256, 256),

            Upsample(scale_factor=2),

            DecoderBlock(256, 128),
            DecoderBlock(128, 128),

            Upsample(scale_factor=2),
            DecoderBlock(128, 64),
            DecoderBlock(64, 64),
        )
        self.output = Sequential(
            Conv2d(64, 1, 1, padding=0),
            Sigmoid()
        )

    def forward(self, x):
        x = self.encoder1(x)
        x = self.encoder_last(x)
        x = self.decoder(x)
        return self.output(x)

In [ ]:
from torch.nn import Module, Sequential, Conv2d, LeakyReLU, Upsample, Sigmoid, MaxPool2d, Tanh, Linear


class Discriminator(Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.main = Sequential(
            Conv2d(4, 3, 1, padding=1),
            LeakyReLU(inplace=True),
            Conv2d(3, 32, 3, padding=1),
            LeakyReLU(inplace=True),
            MaxPool2d(2, stride=2),
            Conv2d(32, 64, 3, padding=1),
            LeakyReLU(inplace=True),
            Conv2d(64, 64, 3, padding=1),
            LeakyReLU(inplace=True),
            MaxPool2d(2, stride=2),
            Conv2d(64, 64, 3, padding=1),
            LeakyReLU(inplace=True),
            Conv2d(64, 64, 3, padding=1),
            LeakyReLU(inplace=True),
            MaxPool2d(2, stride=2))

        self.classifier = Sequential(
            Linear(64 * 32 * 24, 100, bias=True),
            Tanh(),
            Linear(100, 2, bias=True),
            Tanh(),
            Linear(2, 1, bias=True),
            Sigmoid())

    def forward(self, x):
        x = self.main(x)
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)

        return x

### ユーティリティ関数

In [ ]:
from datetime import datetime


def get_timestamp(date_format="{0:%Y%m%d-%H%M%S}"):
    return date_format.format(datetime.now())

In [ ]:
def generate_map(generator: Module, transform, image):
    with torch.no_grad():
        image = transform(image)
        saliency_map = generator(image)
        saliency_map = np.array(saliency_map.cpu())[0, 0]

    saliency_map = saliency_map / saliency_map.sum()
    saliency_map = (())

### トレーニングの実装

In [ ]:
import os
import numpy as np
from datetime import datetime
import torch
from torch.nn import Module, BCELoss
from torch.autograd import Variable, no_grad
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


# Generatorのトレーニングステップ
def generator_train_step(generator, discriminator, valid, image, real_map, criterion, optimizer, alpha=0.005):
    optimizer.zero_grad()
    generated_map = generator(image)

    # Discriminatorへの入力用に元の画像と生成したSaliency Mapを結合して4チャンネルの配列を作る
    fake_d_input = torch.cat((image, generated_map.detach()), 1)

    # Generatorの損失関数を計算
    g_loss1 = criterion(generated_map, real_map)
    g_loss2 = criterion(discriminator(fake_d_input), valid)
    loss = alpha * g_loss1 + g_loss2

    loss.backward()
    optimizer.step()

    return loss.item(), generated_map


# Discriminatorのトレーニングステップ
def discriminator_train_step(discriminator, fake, valid, image, fake_map, real_map, criterion, optimizer):
    optimizer.zero_grad()

    # Discriminatorへの入力用に元の画像と正解データのSaliency Mapを結合して4チャンネルの配列を作る
    real_d_input = torch.cat((image, real_map), 1)
    fake_d_input = torch.cat((image, fake_map), 1)

    # Discriminatorの損失関数を計算
    real_loss = criterion(discriminator(real_d_input), valid)
    fake_loss = criterion(discriminator(fake_d_input), fake)
    loss = (real_loss + fake_loss) / 2

    loss.backward()
    optimizer.step()
    return loss.item()


# トレーニングループ
def training(generator, discriminator, dataloader, criterion, generator_optimizer, discriminator_optimizer, device):
    discriminator_total_loss = 0
    generator_total_loss = 0

    check_directories()

    for batch_index, (image, real_map) in enumerate(dataloader):
        valid = Variable(torch.FloatTensor(image.shape[0], 1).fill_(1.0), requires_grad=False).to(device)
        fake = Variable(torch.FloatTensor(image.shape[0], 1).fill_(0.0), requires_grad=False).to(device)
        image, real_map = image.to(device), real_map.to(device)

        # Generator と Discriminator の学習を交互に実行
        if batch_index % 2 == 0:
            g_loss, fake_map = generator_train_step(generator, discriminator, valid, image, real_map, criterion,
                                                    generator_optimizer)
            generator_total_loss += g_loss
            print(f"Generator Loss: {g_loss:.4f}")
        else:
            d_loss = discriminator_train_step(discriminator, fake, valid, image, fake_map, real_map, criterion,
                                              discriminator_optimizer)
            discriminator_total_loss += d_loss
            print(f"Discriminator Loss: {d_loss:.4f}")


# 重みとログの保存
def save_report(start_time_stamp, generator, discriminator, epoch, save_dir="./logs"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    generator_save_path = f'{os.path.join(save_dir, f"{start_time_stamp}_generator_epoch{epoch}")}.pkl'
    discriminator_save_path = f'{os.path.join(save_dir, f"{start_time_stamp}_discriminator_epoch{epoch}")}.pkl'
    torch.save(generator.state_dict(), generator_save_path)
    torch.save(discriminator.state_dict(), discriminator_save_path)


# テスト（検証）フェーズ
def testing(generator, discriminator, dataloader: DataLoader, criterion, device, show_stride=100):
    generator.to(device)
    discriminator.to(device)

    with torch.no_grad():
        print("validation")
        for i, (image, _) in enumerate(dataloader):
            image = image.to(device)
            generated = generator(image)
            generated_np = np.array(generated.data.cpu())[0, 0]

            plt.imshow(np.array(image[0].cpu()).transpose(1, 2, 0))
            plt.show()
            plt.imshow(generated_np)
            plt.show()
            if i == 1:
                break


# 学習と検証のメインループ
def run(generator, discriminator, train_dataloader, test_dataloader, criterion, generator_optimizer,
        discriminator_optimizer, device, epochs=120):
    time_stamp = get_timestamp()

    for epoch in range(epochs):
        print(f"--- Epoch {epoch + 1}/{epochs} - training start ---")
        training(generator, discriminator, train_dataloader, criterion, generator_optimizer, discriminator_optimizer,
                 device)

        if (epoch % 5 == 0) or (epoch == epochs - 1):
            save_report(time_stamp, generator, discriminator, epoch)

        print(f"--- Epoch {epoch + 1}/{epochs} - testing start ---")
        testing(generator, discriminator, test_dataloader, criterion, device)


### セットアップ と実行

In [ ]:
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.nn import BCELoss
from torch.optim import Adagrad
import torch
from torch.utils.data import DataLoader

# ハイパーパラメータの設定
learning_rate = 0.0001
weight_decay = 3 * learning_rate
epochs = 120

# データの前処理
image_transform = Compose([Resize((192, 256)), ToTensor()])
map_transform = Compose([Resize((192, 256)), ToTensor(), Normalize([0.5], [0.5])])

# データセットの読み込み
salicon_train = SALICONDataset(image_transform=image_transform, map_transform=map_transform)
salicon_test = SALICONDataset(val_mode=True, image_transform=image_transform, map_transform=map_transform)

# データローダーの作成
train_dataloader = DataLoader(salicon_train, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(salicon_test, batch_size=8, shuffle=False, pin_memory=True, num_workers=4)

# モデルの定義
generator = Generator()
discriminator = Discriminator()

# 損失関数とオプティマイザの設定
criterion = BCELoss()
generator_optimizer = Adagrad([
    {'params': generator.encoder_last.parameters()},
    {'params': generator.decoder.parameters()}
], lr=learning_rate, weight_decay=weight_decay)

discriminator_optimizer = Adagrad(discriminator.parameters(), lr=learning_rate, weight_decay=weight_decay)

# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 学習と検証の実行
run(generator, discriminator, train_dataloader, test_dataloader, criterion, generator_optimizer,
    discriminator_optimizer, device, epochs)
